In [1]:
import numpy as np

from keras import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [2]:
# set constants
BATCH_SIZE = 64
EPOCHS = 100
LATENT_DIM = 256
NUM_SAMPLES = 10000
DATA_PATH = 'en-nl.txt'
ENCODING = 'utf-8'

In [3]:
# create variables
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
# load data file
with open(DATA_PATH, 'r', encoding=ENCODING) as f:
    lines = f.read().split('\n')
for line in lines[: min(NUM_SAMPLES, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if (char not in input_characters):
            input_characters.add(char)
    for char in target_text:
        if (char not in target_characters):
            target_characters.add(char)

In [5]:
# sort datalists
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [6]:
# set variables
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 77
Max sequence length for inputs: 20
Max sequence length for outputs: 54


In [8]:
# get token indexes
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
# prepare data for training
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if (t > 0):
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [11]:
# prepare input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(LATENT_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [12]:
# prepare decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
# create model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
# train model
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS, 
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 29s 4ms/step - loss: 1.2635 - val_loss: 1.2948
Epoch 2/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.8873 - val_loss: 0.9841
Epoch 3/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.7236 - val_loss: 0.8621
Epoch 4/100
8000/8000 [==============================] - 28s 4ms/step - loss: 0.6670 - val_loss: 0.8021
Epoch 5/100
8000/8000 [==============================] - 28s 3ms/step - loss: 0.5985 - val_loss: 0.7512
Epoch 6/100
8000/8000 [==============================] - 28s 3ms/step - loss: 0.5585 - val_loss: 0.7185
Epoch 7/100
8000/8000 [==============================] - 30s 4ms/step - loss: 0.5267 - val_loss: 0.6750
Epoch 8/100
8000/8000 [==============================] - 28s 4ms/step - loss: 0.4997 - val_loss: 0.6567
Epoch 9/100
8000/8000 [==============================] - 28s 4ms/step - loss: 0.4760 - val_loss: 0.6350
Epoch 10/100
800

8000/8000 [==============================] - 32s 4ms/step - loss: 0.0743 - val_loss: 0.8597
Epoch 79/100
8000/8000 [==============================] - 32s 4ms/step - loss: 0.0731 - val_loss: 0.8598
Epoch 80/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.0715 - val_loss: 0.8642
Epoch 81/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.0698 - val_loss: 0.8805
Epoch 82/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.0681 - val_loss: 0.8828
Epoch 83/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.0672 - val_loss: 0.8853
Epoch 84/100
8000/8000 [==============================] - 34s 4ms/step - loss: 0.0656 - val_loss: 0.8893
Epoch 85/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.0645 - val_loss: 0.9045
Epoch 86/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.0633 - val_loss: 0.9073
Epoch 87/100
8000/8000 [==============================] - 36s 4ms/st

In [15]:
model.save('en-nl-model.h5')

In [16]:
# build sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [17]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

In [19]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print(f"Input sentence: {input_texts[seq_index]}")
    print(f"Decoded sentence: {decoded_sentence}")

-
Input sentence: Hi.
Decoded sentence: Hé!

-
Input sentence: Hi.
Decoded sentence: Hé!

-
Input sentence: Hi.
Decoded sentence: Hé!

-
Input sentence: Run!
Decoded sentence: Ren!

-
Input sentence: Who?
Decoded sentence: Wie?

-
Input sentence: Wow!
Decoded sentence: Da's niet gek.

-
Input sentence: Fire!
Decoded sentence: Brand!

-
Input sentence: Fire!
Decoded sentence: Brand!

-
Input sentence: Help!
Decoded sentence: Help!

-
Input sentence: Jump.
Decoded sentence: Spring.

-
Input sentence: Stop!
Decoded sentence: Halt!

-
Input sentence: Stop!
Decoded sentence: Halt!

-
Input sentence: Wait.
Decoded sentence: Wacht.

-
Input sentence: Hello!
Decoded sentence: Hoi.

-
Input sentence: Hello!
Decoded sentence: Hoi.

-
Input sentence: Hello!
Decoded sentence: Hoi.

-
Input sentence: I try.
Decoded sentence: Ik probeer het.

-
Input sentence: I won!
Decoded sentence: Ik heb gewonnen!

-
Input sentence: I won.
Decoded sentence: Ik heb gewonnen.

-
Input sentence: Oh no!
Decoded sent